# Nettoyage

## Importation des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

In [ ]:
dataset = pd.read_csv('datastet.csv', nrows=1000)

In [5]:
dataset.head()


,id_stat,nom_region,nom_espece,date_plantation,superficie,pluviometrie,temperature_min,temperature_max,taux_survie_plants,rendement_moyen,annee
0,1,Abidjan,Cacao,2023-04-26,311.35,221.88,23.68,27.66,81.44,9.40,2019
1,1,Abidjan,Cacao,2023-08-20,141.35,221.88,23.68,27.66,81.44,9.40,2019
2,1,Abidjan,Cacao,2024-03-27,301.74,221.88,23.68,27.66,81.44,9.40,2019
3,2,Abidjan,Café,2019-06-26,114.35,92.11,23.94,34.81,75.21,6.29,2019
4,2,Abidjan,Café,2019-12-01,253.65,92.11,23.94,34.81,75.21,6.29,2019


In [8]:
dataset['temperature_min']

0      23.68
1      23.68
2      23.68
3      23.94
4      23.94
       ...  
995    24.12
996    24.12
997    24.12
998    24.12
999    24.12
Name: temperature_min, Length: 1000, dtype: float64

In [17]:
# Transformation de la colonne date_plantation en format datetime
dataset['date_plantation'] = pd.to_datetime(dataset['date_plantation'])
dataset['annee']= dataset['date_plantation'].dt.year

dataset.head()

,id_stat,nom_region,nom_espece,date_plantation,superficie,pluviometrie,temperature_min,temperature_max,taux_survie_plants,rendement_moyen,annee
0,1,Abidjan,Cacao,2023-04-26,311.35,221.88,23.68,27.66,81.44,9.40,2023
1,1,Abidjan,Cacao,2023-08-20,141.35,221.88,23.68,27.66,81.44,9.40,2023
2,1,Abidjan,Cacao,2024-03-27,301.74,221.88,23.68,27.66,81.44,9.40,2024
3,2,Abidjan,Café,2019-06-26,114.35,92.11,23.94,34.81,75.21,6.29,2019
4,2,Abidjan,Café,2019-12-01,253.65,92.11,23.94,34.81,75.21,6.29,2019


In [ ]:
# Sélection des colonnes catégorielles et numériques
categorical_selector = make_column_selector(dtype_exclude=np.number)
numeric_selector = make_column_selector(dtype_include=np.number)

# Transformation des colonnes
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_selector),
    (OneHotEncoder(), categorical_selector)
)

dataset_transformed = preprocessor.fit_transform(dataset)
